# Setup

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# Weekly activity 7

## Question 1

In [2]:
img = cv.imread('winter.jfif', 0)

# histogram equalization
eq = cv.equalizeHist(img)
eq2 = cv.equalizeHist(eq)

cv.imshow('result', np.hstack((img, eq, eq2)))
cv.waitKey(0)
cv.destroyAllWindows()

Comment: There is no difference between first and second output image since it is already equalized. It can't be equalized anymore.

# Question 2.A

In [3]:
img = cv.imread('electronic.jfif', 0)

sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 1)
sobely = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 1)

grad_mag_L2 = cv.magnitude(sobelx, sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

cv.imshow('result', np.hstack((img, grad_mag_L2)))
cv.waitKey(0)
cv.destroyAllWindows()

Comment: Kernel size of 1 is most appropriate to be chosen as it produce result with the minimal noise and there is less white line (noises) in the images compared to kernel size of 3.

# Question 2.B

In [4]:
img_blur = cv.GaussianBlur(img, (5, 5), 0)

sobelx_blur = cv.Sobel(img_blur, cv.CV_64F, 1, 0, ksize = 1)
sobely_blur = cv.Sobel(img_blur, cv.CV_64F, 0, 1, ksize = 1)

grad_mag_L2_blur = cv.magnitude(sobelx_blur, sobely_blur)
grad_mag_L2_blur = cv.convertScaleAbs(grad_mag_L2_blur)

cv.imshow('result', np.hstack((img, grad_mag_L2, grad_mag_L2_blur)))
cv.waitKey(0)
cv.destroyAllWindows()

# Question 2.C

In [5]:
laplacian = cv.Laplacian(img_blur, cv.CV_64F, ksize = 3)

laplacian_8u = cv.convertScaleAbs(laplacian)

cv.imshow('result', laplacian_8u)
cv.waitKey(0)
cv.destroyAllWindows()

Question 2 Conclusion: 
The image processing pathway that is optimal is apply the Gaussian blurring for smoothing the image first, then perform the edge detection using Sobel operator. The reason is that the Gaussian blurring will first reduce the noises in the image first. Then, the edge detection using Sobel operator with kernel size of 1 have the minimal noises in the image processed among all the different techniques applied in Question 2 and the image processed preserves the edge of object better.

# Question 3

In [6]:
img = cv.imread('pineapple.jfif', 0)
blur = cv.GaussianBlur(img, (5, 5), 0)

# Sobel
sobelx = cv.Sobel(blur, cv.CV_64F, 1, 0, ksize = 3)
sobely = cv.Sobel(blur, cv.CV_64F, 0, 1, ksize = 3)
grad_mag_L2 = cv.magnitude(sobelx, sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

# Laplacian
laplacian = cv.Laplacian(blur, cv.CV_64F, ksize = 3)
laplacian_8u = cv.convertScaleAbs(laplacian)

# prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
img_prewittx = cv.filter2D(blur, -1, kernelx)
img_prewitty = cv.filter2D(blur, -1, kernely)

# Scharr
scharrx = cv.Scharr(blur, cv.CV_64F, 1, 0, 3)
scharry = cv.Scharr(blur, cv.CV_64F, 0, 1, 3)
grad_mag_L2_scharr = cv.magnitude(scharrx, scharry)
grad_mag_L2_scharr = cv.convertScaleAbs(grad_mag_L2_scharr)

# Canny
output_canny = cv.Canny(blur, 100, 200, 3)

cv.imshow('result', np.hstack((img, grad_mag_L2, 
                               laplacian_8u, img_prewittx + img_prewitty, 
                               grad_mag_L2_scharr, output_canny)))
cv.waitKey(0)
cv.destroyAllWindows()

Comment on results:
* Sobel: Result with most edges of object detected and edges of object is preserved.
* Laplacian: Edge with similar contrast are hard to be detected.
* Prewitt: Results are similar to Sobel but less edges of object is detected on same colour.
* Scharr derivatives: All the edges in the image is detected and hard to differentiate the object in the image.
* Canny operators: The cleanest result is shown and the cleanest edge is detected.

# Question 4

In [7]:
img = cv.imread('electronic.jfif')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # convert to grayscale

# CLAHE
clahe = cv.createCLAHE(clipLimit = 7.0, tileGridSize = (15, 15))
dst = clahe.apply(gray)

# blur the image then threshold
blur = cv.GaussianBlur(dst, (7, 7), 0)
_, th = cv.threshold(blur, 190, 255, cv.THRESH_BINARY)

# find contour
contour, hierarchy = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# loop the contour to find the laptop contour
for c in contour:
    area = cv.contourArea(c)
    if area >= 900:
        cnt = c
        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
    else:
        continue

cv.imshow('result', img)
cv.waitKey(0)
cv.destroyAllWindows()